In [6]:
import socket
import threading
import json
from flask import Flask, render_template, jsonify
from flask_socketio import SocketIO, emit

app = Flask(__name__)
socketio = SocketIO(app)

track_data = {}

@app.route('/')
def index():
    return render_template('index.html', **track_data)

@app.route('/update', methods=['GET'])
def update():
    return jsonify(track_data)

def update_track_info(json_data):
    global track_data
    track_data = {
        "phrase_type": json_data.get("phraseType"),
        "track_key": json_data.get("trackKey"),
        "track_comment": json_data.get("trackComment"),
        "master_player_number": json_data.get("masterPlayerNumber"),
        "track_time_reached": json_data.get("trackTimereached"),
        "phrase_section": json_data.get("phraseSection"),
        "track_artist": json_data.get("trackArtist"),
        "track_genre": json_data.get("trackGenre"),
        "bpm": json_data.get("bpm"),
        "track_bank": json_data.get("trackBank"),
        "fill": json_data.get("fill"),
        "beat": json_data.get("beat"),
        "track_title": json_data.get("trackTitle"),
        "track_label": json_data.get("trackLabel")
    }
    socketio.emit('update', track_data)

def socket_receiver():
    # Erstellen eines UDP-Sockets
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    port = 7000
    sock.bind(('', port))
    print("Socket verbunden mit Port", port)
    receiving = True

    try:
        while receiving:
            print('\nWarten auf eine Nachricht')
            data, address = sock.recvfrom(4096)
            print('Erhaltene {} Bytes von {}'.format(len(data), address))

            try:
                json_data = json.loads(data)  # Daten dekodieren
                print("JSON-Daten erfolgreich dekodiert:", json_data)
                update_track_info(json_data)
            except json.JSONDecodeError as e:
                print("Fehler beim Dekodieren der JSON-Daten:", e)
    finally:
        sock.close()  # Socket schließen

if __name__ == '__main__':
    receiver_thread = threading.Thread(target=socket_receiver)
    receiver_thread.start()
    socketio.run(app, debug=True, allow_unsafe_werkzeug=True, use_reloader=False)


Werkzeug appears to be used in a production deployment. Consider switching to a production web server instead.
Exception in thread Thread-91 (socket_receiver):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1073, in _bootstrap_inner


 * Serving Flask app '__main__'
 * Debug mode: on


    self.run()
  File "/Users/heikofanieng/Library/Python/3.12/lib/python/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/0m/sqv47k316s5730wvd_tkfwtr0000gn/T/ipykernel_15215/2213331867.py", line 44, in socket_receiver
OSError: [Errno 48] Address already in use
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [20/Jul/2024 22:53:14] "GET / HTTP/1.1" 500 -
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/flask/app.py", line 1498, in __call__
    return self.wsgi_app(environ, start_response)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/flask_socketio/__init__.py", line 43, in __call__
  